# Project: Identify Customer Segments

In this project, you will apply unsupervised learning techniques to identify segments of the population that form the core customer base for a mail-order sales company in Germany. These segments can then be used to direct marketing campaigns towards audiences that will have the highest expected rate of returns. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

This notebook will help you complete this task by providing a framework within which you will perform your analysis steps. In each step of the project, you will see some text describing the subtask that you will perform, followed by one or more code cells for you to complete your work. **Feel free to add additional code and markdown cells as you go along so that you can explore everything in precise chunks.** The code cells provided in the base template will outline only the major tasks, and will usually not be enough to cover all of the minor tasks that comprise it.

It should be noted that while there will be precise guidelines on how you should handle certain tasks in the project, there will also be places where an exact specification is not provided. **There will be times in the project where you will need to make and justify your own decisions on how to treat the data.** These are places where there may not be only one way to handle the data. In real-life tasks, there may be many valid ways to approach an analysis task. One of the most important things you can do is clearly document your approach so that other scientists can understand the decisions you've made.

At the end of most sections, there will be a Markdown cell labeled **Discussion**. In these cells, you will report your findings for the completed section, as well as document the decisions that you made in your approach to each subtask. **Your project will be evaluated not just on the code used to complete the tasks outlined, but also your communication about your observations and conclusions at each stage.**

In [ ]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.preprocessing  import Imputer
from sklearn.impute import SimpleImputer, MissingIndicator

import warnings
warnings.filterwarnings('ignore')

# magic word for producing visualizations in notebook
%matplotlib inline

### Step 0: Load the Data

There are four files associated with this project (not including this one):

- `Udacity_AZDIAS_Subset.csv`: Demographics data for the general population of Germany; 891211 persons (rows) x 85 features (columns).
- `Udacity_CUSTOMERS_Subset.csv`: Demographics data for customers of a mail-order company; 191652 persons (rows) x 85 features (columns).
- `Data_Dictionary.md`: Detailed information file about the features in the provided datasets.
- `AZDIAS_Feature_Summary.csv`: Summary of feature attributes for demographics data; 85 features (rows) x 4 columns

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. You will use this information to cluster the general population into groups with similar demographic properties. Then, you will see how the people in the customers dataset fit into those created clusters. The hope here is that certain clusters are over-represented in the customers data, as compared to the general population; those over-represented clusters will be assumed to be part of the core userbase. This information can then be used for further applications, such as targeting for a marketing campaign.

To start off with, load in the demographics data for the general population into a pandas DataFrame, and do the same for the feature attributes summary. Note for all of the `.csv` data files in this project: they're semicolon (`;`) delimited, so you'll need an additional argument in your [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call to read in the data properly. Also, considering the size of the main dataset, it may take some time for it to load completely.

Once the dataset is loaded, it's recommended that you take a little bit of time just browsing the general structure of the dataset and feature summary file. You'll be getting deep into the innards of the cleaning in the first major step of the project, so gaining some general familiarity can help you get your bearings.

In [ ]:
# Load in the general demographics data.
azdias = pd.read_csv('Udacity_AZDIAS_Subset.csv',sep=';')

# Load in the feature summary file.
feat_info = pd.read_csv('AZDIAS_Feature_Summary.csv',sep=';')

In [ ]:
# Check the structure of the data after it's loaded 
print(azdias.shape)
print(feat_info.shape)

In [ ]:
azdias.head()

In [ ]:
feat_info.head()

> **Tip**: Add additional cells to keep everything in reasonably-sized chunks! Keyboard shortcut `esc --> a` (press escape to enter command mode, then press the 'A' key) adds a new cell before the active cell, and `esc --> b` adds a new cell after the active cell. If you need to convert an active cell to a markdown cell, use `esc --> m` and to convert to a code cell, use `esc --> y`. 

## Step 1: Preprocessing

### Step 1.1: Assess Missing Data

The feature summary file contains a summary of properties for each demographics data column. You will use this file to help you make cleaning decisions during this stage of the project. First of all, you should assess the demographics data in terms of missing data. Pay attention to the following points as you perform your analysis, and take notes on what you observe. Make sure that you fill in the **Discussion** cell with your findings and decisions at the end of each step that has one!

#### Step 1.1.1: Convert Missing Value Codes to NaNs
The fourth column of the feature attributes summary (loaded in above as `feat_info`) documents the codes from the data dictionary that indicate missing or unknown data. While the file encodes this as a list (e.g. `[-1,0]`), this will get read in as a string object. You'll need to do a little bit of parsing to make use of it to identify and clean the data. Convert data that matches a 'missing' or 'unknown' value code into a numpy NaN value. You might want to see how much data takes on a 'missing' or 'unknown' code, and how much data is naturally missing, as a point of interest.

**As one more reminder, you are encouraged to add additional cells to break up your analysis into manageable chunks.**

In [ ]:
# copy dataset and convert to pandas data format
azdias_clean = pd.DataFrame(azdias)
feat_info_clean = pd.DataFrame(feat_info)

In [ ]:
# remove "[]" from missing_or_unknown string and put it in the authentic list
feat_info_clean['missing_or_unknown'] = feat_info_clean['missing_or_unknown'].str.replace('[','').str.replace(']','').str.split(',')
feat_info_clean.set_index(['attribute'], inplace=True)
feat_info_clean.head(5)

In [ ]:
# Identify missing or unknown data values and convert them to NaNs.

for attr in feat_info_clean.index: ## loop through the list of each attribute in feat_info_clean
    
    for x in feat_info_clean.loc[attr]['missing_or_unknown']:  ## loop through the attribute and read the missing_or_unknown value
        
        #if the element list is not empty
        if len(x) > 0:
            # convert the specified value into NaN
            try:
                azdias_clean[attr].replace(int(x), np.nan, inplace=True)
             #the case when the specified value cannot be interpreted as integer
            except:   
                #print(attr, x)
                azdias_clean[attr].replace(x, np.nan, inplace=True)
                print(attr, x)

#### Step 1.1.2: Assess Missing Data in Each Column

How much missing data is present in each column? There are a few columns that are outliers in terms of the proportion of values that are missing. You will want to use matplotlib's [`hist()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.hist.html) function to visualize the distribution of missing value counts to find these columns. Identify and document these columns. While some of these columns might have justifications for keeping or re-encoding the data, for this project you should just remove them from the dataframe. (Feel free to make remarks about these outlier columns in the discussion, however!)

For the remaining features, are there any patterns in which columns have, or share, missing data?

In [ ]:
# Perform an assessment of how much missing data there is in each column of the
# dataset.

# count how many missing values in each column
col_missing = azdias_clean.isnull().sum()  #(by deleting missing value and count it in each column)

col_missing_percentage = col_missing/(azdias.shape[0]) * 100
col_missing_percentage.sort_values(inplace=True) # sorting the values

col_missing_percentage.hist(bins=60)
plt.title("Histogram of misssing values in each column")
plt.xlabel("percentage of missing values (%)")
plt.ylabel("Counts")
plt.show()

In [ ]:
# calculate how many columnes have missing values
col_missing_percentage=pd.DataFrame(col_missing_percentage)
col_missing_percentage.columns = ["Percentage"]

number_of_p = 0
for col in col_missing_percentage["Percentage"]:
        if col > 0:
            number_of_p = number_of_p+1
    
print("There are {} columns with missing values.".format(number_of_p))
#read the percentage of missing values in each columns
col_missing_percentage.sort_values("Percentage",ascending=False).head(61)

In [ ]:
#bar chart of percentage of missing data in each column
col_missing_percentage.plot.bar(figsize=(20,5))
plt.title("Bar chart of misssing values in each column")
plt.ylabel("percentage of missing values (%)")
plt.show()

In [ ]:
# Remove the outlier columns from the dataset. 
del_col = col_missing_percentage[col_missing_percentage['Percentage'] > 30].index
print("There are {} columns with missing values greater than 30%.".format(del_col.shape[0]))
print(del_col)
# drop the column with the missing value greater than 30%
azdias_clean.drop(del_col, axis=1, inplace=True)
azdias_clean.shape

#### Discussion 1.1.2: Assess Missing Data in Each Column

(Double click this cell and replace this text with your own text, reporting your observations regarding the amount of missing data in each column. Are there any patterns in missing values? Which columns were removed from the dataset?)

##### Answer: 
Most of the NaNs in each column is less than 30%. So I removed the columns having NaNs more than 40% from azdias dataset. The 6 columns I removed are 'ALTER_HH', 'GEBURTSJAHR', 'KBA05_BAUMAX', 'KK_KUNDENTYP', 'AGER_TYP','TITEL_KZ'.
       
Patterns of the missing values in the remaining features:
 
- 24 features have no missing values. Most of them are Personality typology.
- The information about people's residential proximity (e.g.the neighbors' family structures) tends to have more missing values.

#### Step 1.1.3: Assess Missing Data in Each Row

Now, you'll perform a similar assessment for the rows of the dataset. How much data is missing in each row? As with the columns, you should see some groups of points that have a very different numbers of missing values. Divide the data into two subsets: one for data points that are above some threshold for missing values, and a second subset for points below that threshold.

In order to know what to do with the outlier rows, we should see if the distribution of data values on columns that are not missing data (or are missing very little data) are similar or different between the two groups. Select at least five of these columns and compare the distribution of values.
- You can use seaborn's [`countplot()`](https://seaborn.pydata.org/generated/seaborn.countplot.html) function to create a bar chart of code frequencies and matplotlib's [`subplot()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.subplot.html) function to put bar charts for the two subplots side by side.
- To reduce repeated code, you might want to write a function that can perform this comparison, taking as one of its arguments a column to be compared.

Depending on what you observe in your comparison, this will have implications on how you approach your conclusions later in the analysis. If the distributions of non-missing features look similar between the data with many missing values and the data with few or no missing values, then we could argue that simply dropping those points from the analysis won't present a major issue. On the other hand, if the data with many missing values looks very different from the data with few or no missing values, then we should make a note on those data as special. We'll revisit these data later on. **Either way, you should continue your analysis for now using just the subset of the data with few or no missing values.**

In [ ]:
# How much data is missing in each row of the dataset?

# count how many missing values in each row
row_missing = azdias_clean.isnull().sum(axis=1) #(by deleting missing value and count it in each row)
row_missing_percentage = row_missing/(len(azdias_clean.columns)) * 100

row_missing_percentage.hist(bins=40)
plt.title("Distribution of number of missing values in each row")
plt.xlabel("percentage of missing values (%)")
plt.ylabel("Number of rows");

In [ ]:
# Write code to divide the data into two subsets based on the number of missing
# values in each row.

row_missing_few = azdias_clean[row_missing_percentage < 20].reset_index(drop=True)
row_missing_more = azdias_clean[row_missing_percentage >= 20].reset_index(drop=True)
#row_missing_few.head() 
print("Divide the data into two subsets with missing values greater or less than 20%.")
print("- One dataset contains {} rows (missing values < 20%)".format(row_missing_few.shape[0]))
print("- Another dataset contains {} rows (missing values >= 20%)".format(row_missing_more.shape[0]))

To check if a threshold of 20% is a fine choice, Kolmogorov–Smirnov test is used here to test each column against the null hypothesis.  The lower the p-value the more we can assume that the two distributions are different.



In [ ]:
from scipy.stats import ks_2samp

def hypothesis_test(df1, df2, cols):
    stats = []
    pvalues = []
    for col in cols:
        counts_main = df1[col].value_counts().sort_index()
        counts_drop = df2[col].value_counts().sort_index()
        try:
            ch = ks_2samp(counts_main, counts_drop)
            stats.append(ch.statistic)
            pvalues.append(ch.pvalue)
        except:
            stats.append(np.nan)
            pvalues.append(np.nan)

    return stats, pvalues

In [ ]:
stats, pvalues = hypothesis_test(row_missing_few, row_missing_more, row_missing_few.columns.values)
comp_df = pd.DataFrame(azdias_clean.columns.values, columns=["features"])
comp_df['stats'] = stats
comp_df['pvalues'] = pvalues
comp_df[comp_df['pvalues']<=0.05].info()

If using 20% missing value as a threshold, 70 features are statistically difference. It is bacause their p-values are less then 5%, and we cannot reject the null hypothesis that the distributions of the two samples are different.

In [ ]:
print('columns without missing data')
col_no_missing = col_missing_percentage[col_missing_percentage['Percentage']==0].index
print(col_no_missing)
print('columns with less missing data (<0.545%)')
col_few_missing = col_missing_percentage[(col_missing_percentage['Percentage']>0) & (col_missing_percentage['Percentage']<0.545)].index
print(col_few_missing)

In [ ]:
# Compare the distribution of values for at least five columns where there are
# no or few missing values, between the two subsets.

def print_countplot(cols,num):
    '''
    function to compare data in two subsets (few missing vs. more missing)
    '''
    fig, axs = plt.subplots(num,2, figsize=(12, 8), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace =0.5 , wspace=.2)
    axs = axs.ravel()

    for i in range(num):
        sns.countplot(row_missing_few[cols[i]], ax=axs[i*2])
        axs[i*2].set_title('few missing')
        
        sns.countplot(row_missing_more[cols[i]], ax=axs[i*2+1])
        axs[i*2+1].set_title('more missing')

In [ ]:
print('Distribution of data values on columns with not missing data')
col_no_missing = ['ZABEOTYP', 'SEMIO_TRADV'] 
print_countplot(col_no_missing,len(col_no_missing))

In [ ]:
# find p value"
print(comp_df[comp_df["features"]==col_no_missing[0]])
print(comp_df[comp_df["features"]==col_no_missing[1]])

In [ ]:
print('Distribution of data values on columns with less missing data')
col_few_missing = ['ALTERSKATEGORIE_GROB', 'GFK_URLAUBERTYP', 'LP_STATUS_GROB'] 
print_countplot(col_few_missing,len(col_few_missing))

In [ ]:
# find p value"
print(comp_df[comp_df["features"]==col_few_missing[0]])
print(comp_df[comp_df["features"]==col_few_missing[1]])
print(comp_df[comp_df["features"]==col_few_missing[2]])

#### Discussion 1.1.3: Assess Missing Data in Each Row

(Double-click this cell and replace this text with your own text, reporting your observations regarding missing data in rows. Are the data with lots of missing values are qualitatively different from data with few or no missing values?)

##### Answers:
- I chose five columns: two columns have no missing value ('ZABEOTYP', 'SEMIO_TRADV'); three columns have few missing values ('ALTERSKATEGORIE_GROB', 'GFK_URLAUBERTYP', 'LP_STATUS_GROB').
- For the columns with no missing values, it appears that both columns have a big difference between few missing data and more missing data in the row. That means those data are special and can not be ignored.
- For the columns with few missing values, ALTERSKATEGORIE_GROB has a little difference between few missing and more missing data, while GFK_URLAUBERTYP and LP_STATUS_GROB show some differences with more data in more_missing data compared to the few missing data. This means that we should pay attension to those data as they are special.
- After I did the statistical analysis using Kolmogorov–Smirnov model, I found all five features are significantly difference between two groups with more missing data and few missing data in each row. Therefore, we should make a note on those data as special.

### Step 1.2: Select and Re-Encode Features

Checking for missing data isn't the only way in which you can prepare a dataset for analysis. Since the unsupervised learning techniques to be used will only work on data that is encoded numerically, you need to make a few encoding changes or additional assumptions to be able to make progress. In addition, while almost all of the values in the dataset are encoded using numbers, not all of them represent numeric values. Check the third column of the feature summary (`feat_info`) for a summary of types of measurement.
- For numeric and interval data, these features can be kept without changes.
- Most of the variables in the dataset are ordinal in nature. While ordinal values may technically be non-linear in spacing, make the simplifying assumption that the ordinal variables can be treated as being interval in nature (that is, kept without any changes).
- Special handling may be necessary for the remaining two variable types: categorical, and 'mixed'.

In the first two parts of this sub-step, you will perform an investigation of the categorical and mixed-type features and make a decision on each of them, whether you will keep, drop, or re-encode each. Then, in the last part, you will create a new data frame with only the selected and engineered columns.

Data wrangling is often the trickiest part of the data analysis process, and there's a lot of it to be done here. But stick with it: once you're done with this step, you'll be ready to get to the machine learning parts of the project!

In [ ]:
# How many features are there of each data type?
feat_info['type'].value_counts()

#### Step 1.2.1: Re-Encode Categorical Features

For categorical data, you would ordinarily need to encode the levels as dummy variables. Depending on the number of categories, perform one of the following:
- For binary (two-level) categoricals that take numeric values, you can keep them without needing to do anything.
- There is one binary variable that takes on non-numeric values. For this one, you need to re-encode the values as numbers or create a dummy variable.
- For multi-level categoricals (three or more values), you can choose to encode the values using multiple dummy variables (e.g. via [OneHotEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)), or (to keep things straightforward) just drop them from the analysis. As always, document your choices in the Discussion section.

In [ ]:
# Assess categorical variables: which are binary, which are multi-level, and
# which one needs to be re-encoded?

binary_category = []
multi_category = []

# features which are categorical
feat_category = feat_info[feat_info['type'] == "categorical"]

# loop thorough each categorical features
for attr in feat_category.index:
    try:
  
    # if the categorical fetures is binary (2 categories)
        if azdias_clean.loc[:,attr].nunique() == 2:
            print("binary with {} values              ".format(azdias_clean.loc[:,attr].dtype), attr)
            binary_category.append(attr)    
    # if the categorical fetures is multi-level (more than 2 categories)
        elif azdias_clean.loc[:,attr].nunique() > 2:
            multi_category.append(attr)
        else:
            print("Categorical Something Wrong!")
    except:
        print ("Not in columns anymore")

In [ ]:
print(multi_category)
print(binary_category)

In [ ]:
azdias_clean1 = row_missing_few

In [ ]:
# Re-encode categorical variable(s) to be kept in the analysis.
# check the binary features whose values are not numeric
print(azdias_clean1["OST_WEST_KZ"].value_counts())

In [ ]:
# Re-encode categorical variable "OST_WEST_KZ" to numeric values

def make_numeric(x):
    if x == 'O':
        return 0
    if x == 'W':
        return 1
    
azdias_clean1['OST_WEST_KZ'] = azdias_clean1['OST_WEST_KZ'].apply(make_numeric)
azdias_clean1["OST_WEST_KZ"].value_counts()

In [ ]:
# drop multi-level category from dataset
azdias_clean1.drop(multi_category, axis=1, inplace=True)

#### Discussion 1.2.1: Re-Encode Categorical Features

(Double-click this cell and replace this text with your own text, reporting your findings and decisions regarding categorical features. Which ones did you keep, which did you drop, and what engineering steps did you perform?)

#### Answers:
- Remove all multi-lavel categorical variables: 'CJT_GESAMTTYP', 'FINANZTYP', 'GFK_URLAUBERTYP', 'LP_FAMILIE_FEIN', 'LP_FAMILIE_GROB', 'LP_STATUS_FEIN', 'LP_STATUS_GROB', 'NATIONALITAET_KZ', 'SHOPPER_TYP', 'ZABEOTYP', 'GEBAEUDETYP', 'CAMEO_DEUG_2015', 'CAMEO_DEU_2015'
- Re-encode categorical variable(s): OST_WEST_KZ. Reengineer it to numeric values.

#### Step 1.2.2: Engineer Mixed-Type Features

There are a handful of features that are marked as "mixed" in the feature summary that require special treatment in order to be included in the analysis. There are two in particular that deserve attention; the handling of the rest are up to your own choices:
- "PRAEGENDE_JUGENDJAHRE" combines information on three dimensions: generation by decade, movement (mainstream vs. avantgarde), and nation (east vs. west). While there aren't enough levels to disentangle east from west, you should create two new variables to capture the other two dimensions: an interval-type variable for decade, and a binary variable for movement.
- "CAMEO_INTL_2015" combines information on two axes: wealth and life stage. Break up the two-digit codes by their 'tens'-place and 'ones'-place digits into two new ordinal variables (which, for the purposes of this project, is equivalent to just treating them as their raw numeric values).
- If you decide to keep or engineer new features around the other mixed-type features, make sure you note your steps in the Discussion section.

Be sure to check `Data_Dictionary.md` for the details needed to finish these tasks.

In [ ]:
# Investigate "PRAEGENDE_JUGENDJAHRE" and engineer two new variables.
def get_decade(x):
    if x in (1,2):
        return 1
    elif x in (3,4):
        return 2
    elif x in (5,6,7):
        return 3
    elif x in (8,9):
        return 4
    elif x in (10,11,12,13):
        return 5
    else:
        return 6
    
def get_movement(x):
    if x in (2,4,6,7,9,11,13,15):
        return 0
    else:
        return 1
    
# create first new varibale: 'PRAEGENDE_JUGENDJAHRE_DECADE'
azdias_clean1['PRAEGENDE_JUGENDJAHRE_DECADE'] = azdias_clean1['PRAEGENDE_JUGENDJAHRE'].apply(get_decade)
# create second new varibale: 'PRAEGENDE_JUGENDJAHRE_MOV'
azdias_clean1['PRAEGENDE_JUGENDJAHRE_MOV'] = azdias_clean1['PRAEGENDE_JUGENDJAHRE'].apply(get_movement)

# drop original column
azdias_clean1=azdias_clean1.drop('PRAEGENDE_JUGENDJAHRE',axis=1)

In [ ]:
# Investigate "CAMEO_INTL_2015" and engineer two new variables.

def wealth(x):
    try:
        return int(x) // 10
    except ValueError:
        return np.nan
    
def life_stage(x):
    try:
        return int(x) % 10
    except ValueError:
        return np.nan

# create first new varibale: 'CAMEO_INTL_2015_WEALTH''
azdias_clean1['CAMEO_INTL_2015_WEALTH'] = azdias_clean1['CAMEO_INTL_2015'].apply(wealth)
# create first new varibale: 'CAMEO_INTL_2015_LIFE_STAGE''
azdias_clean1['CAMEO_INTL_2015_LIFE_STAGE'] = azdias_clean1['CAMEO_INTL_2015'].apply(life_stage)
    
# drop original column
azdias_clean1=azdias_clean1.drop('CAMEO_INTL_2015',axis=1)

In [ ]:
feat_mixed = feat_info[feat_info['type'] == "mixed"]
feat_mixed

In [ ]:
# re-engineer "WOHNLAGE"
# I engineer a second variable that acts as a rural flag (e.g. 1-5 = not rural, 7-8 = rural)

def rural(x):
    if x in (1,2,3,4,5):
        return 0
    else:
        return 1
    
azdias_clean1['WOHNLAGE'] = azdias_clean1['WOHNLAGE'].apply(rural)

In [ ]:
# re-engineer 'PLZ8_BAUMAX'
# The scale is ordinal from 1-4 with size of housing buildings, but 5 indicates business buildings. 
# I replace 5 as a missing value (NaN) from the 'PLZ8_BAUMAX' column

def fillna(x):
    try:
        if x in (1,2,3,4):
            return int(x)
    except ValueError:
        return np.nan
    
azdias_clean1['PLZ8_BAUMAX'] = azdias_clean1['PLZ8_BAUMAX'].apply(fillna)

In [ ]:
# drop 'LP_LEBENSPHASE_FEIN', 'LP_LEBENSPHASE_GROB'
azdias_clean1=azdias_clean1.drop('LP_LEBENSPHASE_FEIN',axis=1)
azdias_clean1=azdias_clean1.drop('LP_LEBENSPHASE_GROB',axis=1)

#### Discussion 1.2.2: Engineer Mixed-Type Features

(Double-click this cell and replace this text with your own text, reporting your findings and decisions regarding mixed-value features. Which ones did you keep, which did you drop, and what engineering steps did you perform?)

##### Answers:
###### Remove: 
1) LP_LEBENSPHASE_FEIN, LP_LEBENSPHASE_GROB
    - According to data dictionary, these two variables represent personal age level, salary level and household type. Other columns already contain these information)
2) KBA05_BAUMAX
    - This feature has been dropped in Step 1.1.2 because it has too many missing values

##### Keep/Re-engineering:  
1) PRAEGENDE_JUGENDJAHRE
    - Engineer two new features to represent decade and movement

2) CAMEO_INTL_2015
    - Engineer two new features to represent wealth and life stage
3) WOHNLAGE
    - Engineer a second variable to represent rural or non-rural 
4) PLZ8_BAUMAX
    - Drop level-5 values from the column, because it indicates business building, whereas 1-4 indicates the family-home building.



#### Step 1.2.3: Complete Feature Selection

In order to finish this step up, you need to make sure that your data frame now only has the columns that you want to keep. To summarize, the dataframe should consist of the following:
- All numeric, interval, and ordinal type columns from the original dataset.
- Binary categorical features (all numerically-encoded).
- Engineered features from other multi-level categorical features and mixed features.

Make sure that for any new columns that you have engineered, that you've excluded the original columns from the final dataset. Otherwise, their values will interfere with the analysis later on the project. For example, you should not keep "PRAEGENDE_JUGENDJAHRE", since its values won't be useful for the algorithm: only the values derived from it in the engineered features you created should be retained. As a reminder, your data should only be from **the subset with few or no missing values**.

In [ ]:
# Do whatever you need to in order to ensure that the dataframe only contains
# the columns that should be passed to the algorithm functions.

azdias_clean1.shape

### Step 1.3: Create a Cleaning Function

Even though you've finished cleaning up the general population demographics data, it's important to look ahead to the future and realize that you'll need to perform the same cleaning steps on the customer demographics data. In this substep, complete the function below to execute the main feature selection, encoding, and re-engineering steps you performed above. Then, when it comes to looking at the customer data in Step 3, you can just run this function on that DataFrame to get the trimmed dataset in a single step.

In [ ]:
def clean_data(df):
    """
    Perform feature trimming, re-encoding, and engineering for demographics
    data
    
    INPUT: Demographics DataFrame
    OUTPUT: Trimmed and cleaned demographics DataFrame
    """
    
    # Put in code here to execute all main cleaning steps:
    # convert missing value codes into NaNs, ...
    feat_info1 = pd.read_csv('AZDIAS_Feature_Summary.csv', delimiter=';')
    feat_info1['missing_or_unknown'] = feat_info1['missing_or_unknown'].str.replace('[','').str.replace(']','').str.split(',')
    feat_info1.set_index(['attribute'], inplace=True)

    for attr in feat_info1.index: 
    
        for x in feat_info1.loc[attr]['missing_or_unknown']:  
            if len(x) > 0:
                try:
                    df[attr].replace(int(x), np.nan, inplace=True)
                except:   
                    df[attr].replace(x, np.nan, inplace=True)
                
    df_clean = pd.DataFrame(df)
    
    ## remove selected columns and rows
    # drop the same columns as in demographics data (sec 1.1.2)
    df_clean.drop(del_col, axis=1, inplace=True) 
    
    # Delete rows with more than 20% of NaNs
    row_missing = df_clean.isnull().sum(axis=1)
    df_clean = df_clean[row_missing/len(df_clean.columns) < 0.2].reset_index(drop=True)
   
    # select, re-encode, and engineer column values.
 
    #Fix binary data
    df_clean['OST_WEST_KZ'] = df_clean['OST_WEST_KZ'].apply(make_numeric)
    
    # Delete categorical data with more than 2 dimensions
    # We know that the binary columns have 3 entries due to some NaN
    
    df_clean['PRAEGENDE_JUGENDJAHRE_DECADE'] = df_clean['PRAEGENDE_JUGENDJAHRE'].apply(get_decade)
    df_clean['PRAEGENDE_JUGENDJAHRE_MOV'] = df_clean['PRAEGENDE_JUGENDJAHRE'].apply(get_movement)
    df_clean['CAMEO_INTL_2015_WEALTH'] = df_clean['CAMEO_INTL_2015'].apply(wealth)
    df_clean['CAMEO_INTL_2015_LIFE_STAGE'] = df_clean['CAMEO_INTL_2015'].apply(life_stage)
    df_clean['WOHNLAGE'] = df_clean['WOHNLAGE'].apply(rural)
    df_clean['PLZ8_BAUMAX'] = df_clean['PLZ8_BAUMAX'].apply(fillna)
    
    # drop original column
    df_clean.drop(multi_category, axis=1, inplace=True)
    df_clean.drop('PRAEGENDE_JUGENDJAHRE',axis=1, inplace=True)
    df_clean.drop('CAMEO_INTL_2015',axis=1, inplace=True)
    
    # drop two columns of mixed-type features
    df_clean.drop('LP_LEBENSPHASE_FEIN',axis=1, inplace=True)
    df_clean.drop('LP_LEBENSPHASE_GROB',axis=1, inplace=True)
    
    # Return the cleaned dataframe.

    return df_clean

In [ ]:
# double check to see if input azdias into the function will return the same shape
azdias = pd.read_csv('Udacity_AZDIAS_Subset.csv',sep=';')
clean_data(azdias).shape

In [ ]:
print(azdias.shape)
print(azdias_clean1.shape)

## Step 2: Feature Transformation

### Step 2.1: Apply Feature Scaling

Before we apply dimensionality reduction techniques to the data, we need to perform feature scaling so that the principal component vectors are not influenced by the natural differences in scale for features. Starting from this part of the project, you'll want to keep an eye on the [API reference page for sklearn](http://scikit-learn.org/stable/modules/classes.html) to help you navigate to all of the classes and functions that you'll need. In this substep, you'll need to check the following:

- sklearn requires that data not have missing values in order for its estimators to work properly. So, before applying the scaler to your data, make sure that you've cleaned the DataFrame of the remaining missing values. This can be as simple as just removing all data points with missing data, or applying an [Imputer](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Imputer.html) to replace all missing values. You might also try a more complicated procedure where you temporarily remove missing values in order to compute the scaling parameters before re-introducing those missing values and applying imputation. Think about how much missing data you have and what possible effects each approach might have on your analysis, and justify your decision in the discussion section below.
- For the actual scaling function, a [StandardScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) instance is suggested, scaling each feature to mean 0 and standard deviation 1.
- For these classes, you can make use of the `.fit_transform()` method to both fit a procedure to the data as well as apply the transformation to the data at the same time. Don't forget to keep the fit sklearn objects handy, since you'll be applying them to the customer demographics data towards the end of the project.

In [ ]:
# consider the process of removing all columns or rows which include NaN in them
print("{}  columns include NaN".format(azdias_clean1.dropna(axis="columns").shape[1]))
print("{} rows include NaN".format(azdias_clean1.dropna(axis="rows").shape[0]))

In [ ]:
# first removes all data points with missing values before feature scaling
azdias_clean2 = azdias_clean1.dropna(axis="columns")
azdias_clean2.shape

Next, I compare three scaling functions in scikit-learn to see if there is any major differences to the transformed dataset when they are applied. 

In [ ]:
def compare_scalers(data, features: list):
    """
    compares three different scaling functions from sklearn
    INPUT: a list of features to be transformed under the different scalers
    Returns: plots of the unscaled data, and the transformed data
    """
    scaler1 = StandardScaler()
    standard_scaled_data = pd.DataFrame(scaler1.fit_transform(data), columns=list(data))
    scaler2 = RobustScaler()
    robust_scaled_data = pd.DataFrame(scaler2.fit_transform(data), columns=list(data))
    scaler3 = MinMaxScaler()
    minmax_scaled_data = pd.DataFrame(scaler3.fit_transform(data), columns=list(data))
    
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(ncols=4, figsize=(20, 5))

    for feature in features:
        ax1.set_title('Before Scaling')
        sns.kdeplot(data[feature], ax=ax1)
        ax2.set_title('After Standard Scaler')
        sns.kdeplot(standard_scaled_data[feature], ax=ax2)
        ax3.set_title('After MinMax Scaler')
        sns.kdeplot(minmax_scaled_data[feature], ax=ax3)
        ax4.set_title('After Robust Scaler')
        sns.kdeplot(robust_scaled_data[feature], ax=ax4)
    plt.show()

In [ ]:
np.random.seed(123)
feature_compared = list(np.random.choice(list(azdias_clean2), size=2, replace=False))
compare_scalers(azdias_clean2, feature_compared)

There are no major qualitative difference in the data after applying these three scalers. So choose anyone of them should be okay. Because we have a large amount of data, we can assume a normal distribution on the data (from the Central Limit Theorem). Based on this assumption, we can go ahead and use the Standard Scaler for feature scaling.

In [ ]:
## Try to fill NaNs with the mean of values in the same column using Imputer
# Apply feature scaling to the general population demographics data.
Imputer = SimpleImputer(strategy= "mean")
scaler = StandardScaler()
azdias_clean_scaled= scaler.fit_transform(Imputer.fit_transform(azdias_clean1))
azdias_clean_scaled = pd.DataFrame(azdias_clean_scaled, columns=list(azdias_clean1))
azdias_clean_scaled.describe()

### Discussion 2.1: Apply Feature Scaling

(Double-click this cell and replace this text with your own text, reporting your decisions regarding feature scaling.)

- I imputed the missing values using Imputer package that replaces the missing value with the mean of the column. 
- To bring the data to same scale for comparison, I standardized the data using StandardScaler package

### Step 2.2: Perform Dimensionality Reduction

On your scaled data, you are now ready to apply dimensionality reduction techniques.

- Use sklearn's [PCA](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) class to apply principal component analysis on the data, thus finding the vectors of maximal variance in the data. To start, you should not set any parameters (so all components are computed) or set a number of components that is at least half the number of features (so there's enough features to see the general trend in variability).
- Check out the ratio of variance explained by each principal component as well as the cumulative variance explained. Try plotting the cumulative or sequential values using matplotlib's [`plot()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.plot.html) function. Based on what you find, select a value for the number of transformed features you'll retain for the clustering part of the project.
- Once you've made a choice for the number of components to keep, make sure you re-fit a PCA instance to perform the decided-on transformation.

In [ ]:
# Apply PCA to the data.
def do_pca(X, n_components):
    '''
    Transforms data using PCA to create n_components, and provides back the results of the
    transformation.

    INPUT: n_components - int - the number of principal components to create
           X - Scaled data

    OUTPUT: pca - the pca object created after fitting the data
            X_pca - the transformed X matrix with new number of components
    '''    
    pca = PCA(n_components)
    X_pca = pca.fit_transform(X)
    return pca, X_pca

def scree_plot(pca):
    '''
    Creates a scree plot associated with the principal components 
    
    INPUT: pca - the result of instantian of PCA in scikit learn
            
    OUTPUT:
            None
    '''
    num_components=len(pca.explained_variance_ratio_)
    ind = np.arange(num_components)
    vals = pca.explained_variance_ratio_
 
    plt.figure(figsize=(15, 5))
    ax = plt.subplot(111)
    cumvals = np.cumsum(vals)
    ax.bar(ind, vals)
    ax.plot(ind, cumvals)
    for i in range(num_components):
        ax.annotate(r"%s%%" % ((str(vals[i]*100)[:4])), (ind[i]+0.2, vals[i]), va="bottom", ha="center", fontsize=12)
 
    ax.xaxis.set_tick_params(width=0)
    ax.yaxis.set_tick_params(width=2, length=12)
 
    ax.set_xlabel("Principal Component")
    ax.set_ylabel("Explained Variance (%)")
    plt.title('Explained Variance Per Principal Component')

In [ ]:
# Investigate the variance accounted for by each principal component.

pca, X_pca = do_pca(azdias_clean_scaled, azdias_clean1.shape[1])
scree_plot(pca)

In [ ]:
min_n_components = min(np.where(np.cumsum(pca.explained_variance_ratio_)>0.85)[0]+1)

print('We retain {} components because they explain 85% variance in the data'.format(min_n_components));

In [ ]:
# Re-apply PCA to the data while selecting for number of components to retain.

pca27, X_pca = do_pca(azdias_clean_scaled, min_n_components)
scree_plot(pca27)

### Discussion 2.2: Perform Dimensionality Reduction

(Double-click this cell and replace this text with your own text, reporting your findings and decisions regarding dimensionality reduction. How many principal components / transformed features are you retaining for the next step of the analysis?)

##### Answer:
The scree plot above shows that after using 27 components, the variance explained percentage starts to level off and converge towards 85%. That seems to be a good number of components to retain.

### Step 2.3: Interpret Principal Components

Now that we have our transformed principal components, it's a nice idea to check out the weight of each variable on the first few components to see if they can be interpreted in some fashion.

As a reminder, each principal component is a unit vector that points in the direction of highest variance (after accounting for the variance captured by earlier principal components). The further a weight is from zero, the more the principal component is in the direction of the corresponding feature. If two features have large weights of the same sign (both positive or both negative), then increases in one tend expect to be associated with increases in the other. To contrast, features with different signs can be expected to show a negative correlation: increases in one variable should result in a decrease in the other.

- To investigate the features, you should map each weight to their corresponding feature name, then sort the features according to weight. The most interesting features for each principal component, then, will be those at the beginning and end of the sorted list. Use the data dictionary document to help you understand these most prominent features, their relationships, and what a positive or negative value on the principal component might indicate.
- You should investigate and interpret feature associations from the first three principal components in this substep. To help facilitate this, you should write a function that you can call at any time to print the sorted list of feature weights, for the *i*-th principal component. This might come in handy in the next step of the project, when you interpret the tendencies of the discovered clusters.

In [ ]:
# Map weights for the first principal component to corresponding feature names
# and then print the linked values, sorted by weight.
# HINT: Try defining a function here or in a new cell that you can reuse in the
# other cells.

def map_weights(pca, cols, component):
    comp =  pd.DataFrame({'attribute': cols, 'weight': pca.components_[component], 'weight_abs': np.abs(pca.components_[component])})
    return comp.sort_values(by=['weight_abs'], ascending=False)

In [ ]:
comp_0 = map_weights(pca27, azdias_clean1.columns, 0)
comp_0.head(6)

In [ ]:
# Map weights for the second principal component to corresponding feature names
# and then print the linked values, sorted by weight.

comp_1 = map_weights(pca27, azdias_clean1.columns, 1)
comp_1.head(6)

In [ ]:
# Map weights for the third principal component to corresponding feature names
# and then print the linked values, sorted by weight.

comp_2 = map_weights(pca27, azdias_clean1.columns, 2)
comp_2.head(6)

In [ ]:
comp_3 = map_weights(pca27, azdias_clean1.columns, 3)
comp_3.head(6)

### Discussion 2.3: Interpret Principal Components

(Double-click this cell and replace this text with your own text, reporting your observations from detailed investigation of the first few principal components generated. Can we interpret positive and negative values from them in a meaningful way?)

##### Answers:

Here are the interpretation of the first three components as requested above:

##### 1st Component (0):  mainly about the residential informaton (PLZ8 and RR3 region).

+	MOBI_REGIO	: Movement patterns
+	PLZ8_ANTG3	: Number of 6-10 family houses in the PLZ8 region
+	PLZ8_ANTG1	: Number of 1-2 family houses in the PLZ8 region
+	PLZ8_ANTG4	: Number of 10+ family houses in the PLZ8 region
+	KBA05_ANTG1	 :  Number of 1-2 family houses in the microcell
+	KBA05_GBZ	: Number of buildings in the microcell

##### 2nd Component (1):  mainly about the personality and finacial informaton .
+ ALTERSKATEGORIE_GROB	: Estimated age based on given name analysis
+	SEMIO_REL	: Personality typology,religious
+	PRAEGENDE_JUGENDJAHRE_DECADE : Dominating movement of person's youth 
+  FINANZ_SPARER	: Financial typology, money-saver
+ FINANZ_VORSORGER	: Financial typology, be prepared
+	SEMIO_PFLICHT	: Personality typology, dutiful


##### 3rd Component (2):  similar to 2nd component, mainly about the personality informaton.

+	ANREDE_KZ	: Gender
+	SEMIO_VERT	:Personality typology, dreamful
+	SEMIO_KAEM	:Personality typology, combative attitude
+	SEMIO_DOM	:Personality typology, dominant-minded
+	SEMIO_KRIT	:Personality typology, critical-minded
+	SEMIO_SOZ	:Personality typology, socially-minded


## Step 3: Clustering

### Step 3.1: Apply Clustering to General Population

You've assessed and cleaned the demographics data, then scaled and transformed them. Now, it's time to see how the data clusters in the principal components space. In this substep, you will apply k-means clustering to the dataset and use the average within-cluster distances from each point to their assigned cluster's centroid to decide on a number of clusters to keep.

- Use sklearn's [KMeans](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans) class to perform k-means clustering on the PCA-transformed data.
- Then, compute the average difference from each point to its assigned cluster's center. **Hint**: The KMeans object's `.score()` method might be useful here, but note that in sklearn, scores tend to be defined so that larger is better. Try applying it to a small, toy dataset, or use an internet search to help your understanding.
- Perform the above two steps for a number of different cluster counts. You can then see how the average distance decreases with an increasing number of clusters. However, each additional cluster provides a smaller net benefit. Use this fact to select a final number of clusters in which to group the data. **Warning**: because of the large size of the dataset, it can take a long time for the algorithm to resolve. The more clusters to fit, the longer the algorithm will take. You should test for cluster counts through at least 10 clusters to get the full picture, but you shouldn't need to test for a number of clusters above about 30.
- Once you've selected a final number of clusters to use, re-fit a KMeans instance to perform the clustering operation. Make sure that you also obtain the cluster assignments for the general demographics data, since you'll be using them in the final Step 3.3.

In [ ]:
# Over a number of different cluster counts...

scores = []

#test to determine k
for k in range(1,30,2):

    # run k-means clustering on the data
    kmeans = KMeans(k, random_state=42)
    model = kmeans.fit(X_pca)
    
    # compute the average within-cluster distances.
    score = np.abs(kmeans.score(X_pca)) 
    scores.append(score)  

    print("Checked = {}, Score = {}".format(k,score))

In [ ]:
# Investigate the change in within-cluster distance across number of clusters.
# HINT: Use matplotlib's plot function to visualize this relationship.

plt.plot(np.arange(1, 30, 2), scores, linestyle="--", marker='o', color='b')
plt.title("Number of Clusters vs Average Distance to Cluster Centers")
plt.xlabel("number of clusters, k")
plt.ylabel("average distance");

In [ ]:
# Re-fit the k-means model with the selected number of clusters and obtain
# cluster predictions for the general population demographics data.

# instantiate the kmean object(k=10)
kmeans10= KMeans(10, random_state=42)

# fit the kmeans model with X_pca (THIS WOULD BE USED LATER)
# and also get the clustering result labels

demo_model = kmeans10.fit(X_pca)
demo_predict = demo_model.predict(X_pca)

### Discussion 3.1: Apply Clustering to General Population

(Double-click this cell and replace this text with your own text, reporting your findings and decisions regarding clustering. Into how many clusters have you decided to segment the population?)

I visualized each k (1 to 29)'s score, which is a very time consuming task. I have seen the marginal drop in distance is diminishing at around 10 cluster. So I decided to move forward with 10 clusters. 

By applying k-means clustering to the demographic data, I classified each people and toggled 1 of 10 labels. 

### Step 3.2: Apply All Steps to the Customer Data

Now that you have clusters and cluster centers for the general population, it's time to see how the customer data maps on to those clusters. Take care to not confuse this for re-fitting all of the models to the customer data. Instead, you're going to use the fits from the general population to clean, transform, and cluster the customer data. In the last step of the project, you will interpret how the general population fits apply to the customer data.

- Don't forget when loading in the customers data, that it is semicolon (`;`) delimited.
- Apply the same feature wrangling, selection, and engineering steps to the customer demographics using the `clean_data()` function you created earlier. (You can assume that the customer demographics data has similar meaning behind missing data patterns as the general demographics data.)
- Use the sklearn objects from the general demographics data, and apply their transformations to the customers data. That is, you should not be using a `.fit()` or `.fit_transform()` method to re-fit the old objects, nor should you be creating new sklearn objects! Carry the data through the feature scaling, PCA, and clustering steps, obtaining cluster assignments for all of the data in the customer demographics data.

In [ ]:
# Load in the customer demographics data.
cust =  pd.read_csv('Udacity_CUSTOMERS_Subset.csv', delimiter=';')

In [ ]:
# clean the customer demographics data.
cust_clean = clean_data(cust)
cust_clean.head()

In [ ]:
print("shape of customers' data before cleaning :",cust.shape)
print("shape of customers' data after cleaning :",cust_clean.shape)
print("shape of demographic data after cleaning :",azdias_clean1.shape)

In [ ]:
# compare the number of column between demographic data and feature data
azdias_clean1.columns[~azdias_clean1.columns.isin(cust_clean.columns)]

customer data and demographic data have the same numbers of feature

In [ ]:
# Apply preprocessing, feature transformation, and clustering from the general
# demographics onto the customer data, obtaining cluster predictions for the
# customer demographics data.

# Standardize or Normalize
cust_clean_scaled = scaler.transform(Imputer.fit_transform(cust_clean))
cust_clean_scaled = pd.DataFrame(cust_clean_scaled, columns=list(cust_clean))

# PCA
cust_clean_pca = pd.DataFrame(pca27.transform(cust_clean_scaled))

# predict (use kmean to predict pca data)
cust_clean_predict = kmeans10.predict(cust_clean_pca);

In [ ]:
# check mean is not so far from 0 as well as std not from 1
cust_clean_scaled.describe()

### Step 3.3: Compare Customer Data to Demographics Data

At this point, you have clustered data based on demographics of the general population of Germany, and seen how the customer data for a mail-order sales company maps onto those demographic clusters. In this final substep, you will compare the two cluster distributions to see where the strongest customer base for the company is.

Consider the proportion of persons in each cluster for the general population, and the proportions for the customers. If we think the company's customer base to be universal, then the cluster assignment proportions should be fairly similar between the two. If there are only particular segments of the population that are interested in the company's products, then we should see a mismatch from one to the other. If there is a higher proportion of persons in a cluster for the customer data compared to the general population (e.g. 5% of persons are assigned to a cluster for the general population, but 15% of the customer data is closest to that cluster's centroid) then that suggests the people in that cluster to be a target audience for the company. On the other hand, the proportion of the data in a cluster being larger in the general population than the customer data (e.g. only 2% of customers closest to a population centroid that captures 6% of the data) suggests that group of persons to be outside of the target demographics.

Take a look at the following points in this step:

- Compute the proportion of data points in each cluster for the general population and the customer data. Visualizations will be useful here: both for the individual dataset proportions, but also to visualize the ratios in cluster representation between groups. Seaborn's [`countplot()`](https://seaborn.pydata.org/generated/seaborn.countplot.html) or [`barplot()`](https://seaborn.pydata.org/generated/seaborn.barplot.html) function could be handy.
  - Recall the analysis you performed in step 1.1.3 of the project, where you separated out certain data points from the dataset if they had more than a specified threshold of missing values. If you found that this group was qualitatively different from the main bulk of the data, you should treat this as an additional data cluster in this analysis. Make sure that you account for the number of data points in this subset, for both the general population and customer datasets, when making your computations!
- Which cluster or clusters are overrepresented in the customer dataset compared to the general population? Select at least one such cluster and infer what kind of people might be represented by that cluster. Use the principal component interpretations from step 2.3 or look at additional components to help you make this inference. Alternatively, you can use the `.inverse_transform()` method of the PCA and StandardScaler objects to transform centroids back to the original data space and interpret the retrieved values directly.
- Perform a similar investigation for the underrepresented clusters. Which cluster or clusters are underrepresented in the customer dataset compared to the general population, and what kinds of people are typified by these clusters?

In [ ]:
# Compare the proportion of data in each cluster for the customer data to the
# proportion of data in each cluster for the general population.

plt.figure(1, figsize=(12, 3))
plt.subplot(121)
sns.countplot(demo_predict)
plt.xlabel('Cluster')
plt.title('The cluster distribution of general population data')
plt.subplot(122)
sns.countplot(cust_clean_predict)
plt.xlabel('Cluster')
plt.title('The cluster distribution of customer data')
plt.show()


ax = plt.figure().add_axes([0,0,1.8,0.5], title='Demographic vs Customer distribution plot')

sns.distplot(demo_predict, label='demographic dara',ax=ax)
sns.distplot(cust_clean_predict, label='customer data', ax=ax)
plt.xlabel('count')
ax.legend();

In [ ]:
# Compare the proportion of data in each cluster for the customer data to the
# proportion of data in each cluster for the general population.

# calculate the proportion of customer data from in each cluster 
prop_cust = pd.DataFrame(cust_clean_predict,columns=['Cluster']).reset_index().groupby('Cluster').count()/cust_clean_predict.shape[0]*100
# calculate the proportion of general population data in each cluster 
prop_pop = pd.DataFrame(demo_predict,columns=['Cluster']).reset_index().groupby('Cluster').count()/demo_predict.shape[0]*100
diff = prop_cust - prop_pop

diff.rename_axis({'index':'Difference'}, axis=1, inplace=True)
prop_cust.rename_axis({'index':'Prop_customer'}, axis=1, inplace=True)
prop_pop.rename_axis({'index':'Prop_population'}, axis=1, inplace=True)
diff = diff.join(prop_cust).join(prop_pop).sort_values('Difference',ascending=True)

diff

Apply the inverse transform only through the PCA transform. We can take a look at the magnitudes of the scaled coefficients to interpret the values. Since StandardScaler was used, strongly negative values suggest lower values on the original scale and positive values suggest higher values on the original scale.

In [ ]:
def plot_scaled_comparison(df_sample, kmeans, cluster):
    X = pd.DataFrame.from_dict(dict(zip(df_sample.columns,
pca27.inverse_transform(kmeans.cluster_centers_[cluster]))), orient='index').rename(
columns={0: 'feature_values'}).sort_values('feature_values', ascending=False)
    X['feature_values_abs'] = abs(X['feature_values'])
    pd.concat((X['feature_values'][:10], X['feature_values'][-10:]), axis=0).plot(kind='barh');

In [ ]:
# What kinds of people are part of a cluster that is overrepresented in the
# customer data compared to the general population?
print("overrepresented clusters")
#overrepresented Cluster 6 (11.3 %)
c6 = pd.DataFrame()
c6['Component'] = list(range(0,27))
c6['Center'] = list(kmeans10.cluster_centers_[6])
print("Cluster 6")
print(c6.sort_values('Center',ascending=False).head(6))

# overrepresented Cluster 9 (7.98 %)
c9 = pd.DataFrame()
c9['Component'] = list(range(0,27))
c9['Center'] = list(kmeans10.cluster_centers_[9])
print("Cluster 9")
print(c9.sort_values('Center',ascending=False).head(6))

In [ ]:
plot_scaled_comparison(cust_clean, kmeans10, 6)

In [ ]:
plot_scaled_comparison(cust_clean, kmeans10, 9)

In [ ]:
# What kinds of people are part of a cluster that is underrepresented in the
# customer data compared to the general population?
print("underrepresented clusters")
#underrepresented Cluster 2 (-9.41 %)
c2 = pd.DataFrame()
c2['Component'] = list(range(0,27))
c2['Center'] = list(kmeans10.cluster_centers_[2])
print("Cluster 2")
print(c2.sort_values('Center',ascending=False).head(6))

#underrepresented Cluster 7 (-9.41 %)
c7 = pd.DataFrame()
c7['Component'] = list(range(0,27))
c7['Center'] = list(kmeans10.cluster_centers_[7])
print("Cluster 7")
print(c7.sort_values('Center',ascending=False).head(6))

In [ ]:
plot_scaled_comparison(cust_clean, kmeans10, 2)

In [ ]:
plot_scaled_comparison(cust_clean, kmeans10, 7)

### Discussion 3.3: Compare Customer Data to Demographics Data

(Double-click this cell and replace this text with your own text, reporting findings and conclusions from the clustering analysis. Can we describe segments of the population that are relatively popular with the mail-order company, or relatively unpopular with the company?)

#### Conclusion: 
The proportion of data in cluster 6 and 9 is significantly higher in the customer data, while the propotion of data in cluster 2 and 7 is significantly lower in the customer data, comparing to the demographic/population data. 

###### Over-represented clusters: (Future target customers)
*Cluster-6* : This group is over-representative in our customer datasbase (33.7%) compared to the general population (7.6%). Most of people in this group are male. This group tends to have less dreamful and socially-mindedand. They have more combative attitude, dominant-minded, and critical-minded. (compnent 2). Most of people in this group have membersship of green avantgarde in environmental sustainability. They also tend to have dominating movement of person's youth. They tend to have higher density of households per square kilometer (component 3).

*Cluster-9* : Meanwhile, they have lower or average purchasing power in RR1 region. They are most likely lower middle class, tranditional worker or in marginal groups. They have lower share of 3-5 family homes. (component 6)

###### Under-represented clusters: (Not our future target customers)
*Cluster-2* : This group is under-representative in our customer datasbase (1.56%) compared to the general population (10.97%). Their insurance Insurance typology tend to be social-safety driven. Their health typology tend to be classification not possible or critical reseraved. They have less materialistic and rational personality (component 8). Their insurance typology tend to have more individualistic-accepting risks. They are likely to be conservative low-returners or determined minimal-Returners. They have less sensual-minded personality (component 23).

*Cluster-7* : People in this group have high movement patterns. They have higher or average share of 6-10 family homes, lower share of 1-2 family homes, but high share of 10+ family homes in PLZ8 region. (component 0)


> Congratulations on making it this far in the project! Before you finish, make sure to check through the entire notebook from top to bottom to make sure that your analysis follows a logical flow and all of your findings are documented in **Discussion** cells. Once you've checked over all of your work, you should export the notebook as an HTML document to submit for evaluation. You can do this from the menu, navigating to **File -> Download as -> HTML (.html)**. You will submit both that document and this notebook for your project submission.